# G$^2$Pnet: The Grassmanian Gaussian Process Network
We check the implementation of G2PnetSolver (ver 4.) class in ../src/

Created by Tianpei Xie, 04/12/2016

Last Edit 04/13/2016

In [1]:
%reset
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics.pairwise import rbf_kernel, pairwise_kernels
import GPy
import GPy.util as gutil

%matplotlib inline 
plt.close('all')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


/Applications/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
Ifsavefig =  True
Ifsavedata = False #True
trial = 9

In [3]:
G=nx.read_adjlist("../data/04_01_2016_experiment8.adjlist")

size = 40
color_map = {0:'r', 1:'c', 2:'y'}
#center = np.zeros(2)
#center[1] = 1
center = None

n = 40
m = 0
G0= nx.barbell_graph(n,m) 
#generate the bar-bell graph, which consists of two fully connected component and a path connecting them 

label0 = np.zeros([2*n,1])
label0[n:2*n] = np.ones([n,1])
#### labeling of the cluster
for node in G0.nodes():
    G0.node[node]['category'] = int(label0[node,0])

In [4]:
data_df = pd.read_csv("../data/04_01_2016_experiment8_X.csv", index_col = 'ID')
spectral_df = pd.read_csv("../data/04_01_2016_experiment8_U.csv", index_col ='ID')
U_init_pd = pd.read_csv("../data/04_01_16_8_U_init.csv", index_col = 0)

In [5]:
X = data_df.loc[:,['0_cos','1_cos']].values
U = spectral_df.loc[:,['PX','PY']].values
labels = data_df.loc[:,'LABEL'].values
U_init = U_init_pd.loc[:,['0','1']].values

In [6]:
nsample, ndim = X.shape
_ , ndim_latent = U.shape

In [7]:
import sys
import os
dirorg = os.path.dirname(os.getcwd())
sys.path.insert(0, dirorg+"/src/")

In [8]:
import G2PnetSolver_v5 #G2PnetSolver_v2
#from imp import reload
#reload(G2PnetSolver_v2)

In [9]:
from G2PnetSolver_v5 import  G2PnetSolver as Solver
from G2PnetSolver_v5 import  Grassmann_update, check_step

In [10]:
choice = "near"
IfGen = False

In [11]:
if IfGen:
    np.random.seed(seed=150)

    if choice == "random":
        print("Random Initialization")
        Temp = np.random.randn(nsample, nsample)
        Temp_sym = (Temp + Temp.T) / 2 
        _, U_temp = np.linalg.eig(Temp_sym)
        U_test = U_temp[:, np.arange(ndim_latent)]
        U0 = U_test
    elif choice == "near":
        print("Initialization in neigborhood.")
        U0 = U

    eta0 = 0.75
    np.random.seed(seed=160)
    G_temp = np.random.randn(nsample, ndim_latent) / (np.sqrt(nsample * ndim_latent))
    H_temp = -np.dot((np.eye(nsample) - gutil.linalg.tdot(U0)),G_temp)
    print(str(check_step(H_temp, eta0)))
    while not check_step(H_temp, eta0):
        np.random.seed(seed=160)
        G_temp = np.random.randn(nsample, ndim_latent) / np.sqrt(nsample * ndim_latent)
        H_temp = -np.dot((np.eye(nsample) - gutil.linalg.tdot(U0)),G_temp)
    U_init = Grassmann_update(U0, H_temp, eta=eta0)

In [12]:
if Ifsavedata:
    U_columns = [str(i) for i in np.arange(ndim_latent)]
    U_df = pd.DataFrame(data = U_init, columns=U_columns, dtype= float)
    if choice == "random":
        U_df['RANDOM'] = np.ones([nsample, 1])
    elif choice == "near":
        U_df['RANDOM'] = np.zeros([nsample, 1])
    
    U_df.to_csv("../data/04_12_16_" + str(trial)  + "_U_init.csv")

In [ ]:
add_reg = True
lambda_array = np.array([100]) #10, 1, 0.001
eta = 1e-5 #0.0001
max_iters = 600
kern_array = [GPy.kern.RBF(2) for i in np.arange(len(lambda_array))]
kern_grass_array = [GPy.kern.RBF(2) for i in np.arange(len(lambda_array))]
solver_array = []
hist_nll_eu_array = []
hist_nll_grass_array = []
hist_Hsig_array = []

In [ ]:
fig= plt.figure(1)
fig.set_size_inches(12.0, 30.0)
plt.style.use('seaborn-white')

for i, lambda_var in enumerate(lambda_array): 
    ax = plt.subplot(5,1,i+1)
    solver = Solver(X=X, U=U_init, kernel=kern_array[i], U_ref= U, add_reg = add_reg, lambda_var = lambda_var, eta=eta)
    hist_nll_eu, hist_eta, hist_kern_eu = solver.optimize(max_iters = max_iters)
    solver_array.append(solver)
    hist_nll_eu_array.append(hist_nll_eu)
    
    
    solver2 = Solver(X=X, U=U_init, kernel=kern_grass_array[i], U_ref = U, add_reg = add_reg, lambda_var = lambda_var, eta=eta)
    hist_nll_grass, hist_eta, hist_kern_grass = solver2.optimize(max_iters = max_iters, optimizor = "grad_descent_grass")
    hist_Hsig_array.append(solver2.hist_Hsig)
    
    solver_array.append(solver2)
    hist_nll_grass_array.append(hist_nll_grass)
        
    %xdel solver
    %xdel solver2

kernel initialization ...
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i     f              |g|        
    00s02  005   3.050193e+02   4.408648e+01 
    00s05  014   1.509605e+02   3.865785e-02 
    00s06  017   1.509592e+02   5.131985e-08 
    00s06  018   1.509592e+02   5.131985e-08 
Runtime:     00s06
Optimization status: Converged

negative log-likelihood compute
[step   0] neg-log-likelihood 1102.420586 stepsize 0.0000
The gradient of L w.r.t. U not been computed. Just compute it anyway.
The gradient of L w.r.t K not been computed. Just compute it anyway.
The gradient of K w.r.t. U not been computed. Just compute it anyway
[step   1] neg-log-likelihood 947.221737 stepsize 0.0000
[step   2] neg-log-likelihood 834.795294 stepsize 0.0000
[step   3] neg-log-likelihood 783.584312 stepsize 0.0000
[step   4] neg-log-likelihood 1906.458051 stepsize 0.0000
[step   5] neg-log-likelihood 1957.654065 stepsize 0.0000
[step   6] neg-log-likelihood 2007.046700 stepsize 0.0000
[step

In [ ]:
fig= plt.figure(1)
fig.set_size_inches(12.0, 6.0*len(lambda_array))
plt.style.use('seaborn-white')

for i in np.arange(len(lambda_array)):
    ax = plt.subplot(len(lambda_array),1,i+1)
    h1, = plt.plot(np.arange(max_iters), hist_nll_eu_array[i], 'b')
    h2, = plt.plot(np.arange(max_iters), hist_nll_grass_array[i], 'r')
    if add_reg:
        ax.legend([h1, h2], [r"Euclidean ($\eta$ = {0:6.5f}, $\lambda$ = {1:5.3f})".format(eta, lambda_array[i]), \
                             r"Grassmann ($\eta$ = {0:6.5f}, $\lambda$ = {1:5.3f})".format(eta, lambda_array[i])], \
                             fontsize = 15, loc='upper center', frameon=True)
    else:    
        ax.legend([h1, h2], [r"Euclidean ($\eta$ = {0:6.5f})".format(eta), r"Grassmann ($\eta$ = {0:5.4f})".format(eta)], \
                             fontsize = 15, loc='upper center', frameon=True)
    plt.grid(1)
    plt.ylabel('negative log-likelihood', fontsize = 15)
    plt.xlabel('iterations', fontsize = 15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
plt.show    
if Ifsavefig:
    file_org = "../figures/04_12_16_" + str(trial) + "_"
    file_tail = "_nll_iters_Grass_Eu.eps"
    if choice == "random":
        file_comment = "rand"
    elif choice == "near":
        file_comment = "near"
        
    if add_reg:
        file_comment = file_comment +'Reg'
        
    fig.savefig(file_org + file_comment + file_tail )  

In [ ]:
fig= plt.figure(2)
fig.set_size_inches(12.0, 6.0*len(lambda_array))
plt.style.use('seaborn-white')

for i in np.arange(len(lambda_array)):
    ax = plt.subplot(len(lambda_array),1,i+1)
    h1, = plt.plot(np.arange(max_iters), hist_Hsig_array[i][:,0], 'b')
    h2, = plt.plot(np.arange(max_iters), hist_Hsig_array[i][:,1], 'r')
    if add_reg:
        ax.legend([h1, h2], [r"1st singular value ($\eta$ = {0:6.5f}, $\lambda$ = {1:5.3f})".format(eta, lambda_array[i]), \
                             r"2nd singular value ($\eta$ = {0:6.5f}, $\lambda$ = {1:5.3f})".format(eta, lambda_array[i])], \
                             fontsize = 15, loc='upper right', frameon=True)
    else:    
        ax.legend([h1, h2], [r"1st singular value ($\eta$ = {0:6.5f})".format(eta), r"2nd singular value ($\eta$ = {0:5.4f})".format(eta)], \
                             fontsize = 15, loc='upper right', frameon=True)
    plt.grid(1)
    plt.ylabel('singular value', fontsize = 15)
    plt.xlabel('iterations', fontsize = 15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
plt.show    
if Ifsavefig:
    file_org = "../figures/04_12_16_" + str(trial) + "_"
    file_tail = "_sig_iters_Grass_Eu.eps"
    if choice == "random":
        file_comment = "rand"
    elif choice == "near":
        file_comment = "near"
        
    if add_reg:
        file_comment = file_comment +'Reg'
        
    fig.savefig(file_org + file_comment + file_tail )  